#XNAT download and CUDA-powered SMIT rectal segmentation

##Install python and system prerequisites

In [ ]:
import os

In [ ]:
%%capture
!pip install pyxnat
!pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git"
!pip install sevenbridges-python

##Download T2 sequence from XNAT

In [ ]:
from pyxnat import Interface
from glob import glob
import shutil


def getXNATData(xhost,user,scandict,downloadDir):
  xnat = Interface(xhost, user)
  os.makedirs(downloadDir, exist_ok=True)
  expdirlist = []
  for scan_entry in scandict:
    proj = scan_entry['exam']['proj']
    subj = scan_entry['exam']['subj']
    exp = scan_entry['exam']['exp']
    scan_list = scan_entry['exam']['scan_list']
    expdir = os.path.join(downloadDir,exp)
    expdirlist.append(expdir)
    os.makedirs(expdir, exist_ok = True)
    xexp = xnat.select.project(proj).subject(subj).experiment(exp)
    for scan in scan_list:
      xres = xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('DICOM')
      if not xres.exists():
        xres = xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('secondary')
      xres.get(downloadDir,extract=True)
    for dcmfolder in ['DICOM','secondary']:
      dcmlist = glob(os.path.join(downloadDir,dcmfolder,'*.dcm'))
      print(dcmlist)
      for dcm in dcmlist:
        shutil.move(dcm, expdir)
  for dcmfolder in ['DICOM','secondary']:
    if os.path.exists(os.path.join(downloadDir,dcmfolder)):
      os.rmdir(os.path.join(downloadDir,dcmfolder))
    if os.path.exists(os.path.join(downloadDir,dcmfolder + '.zip')):
      os.remove(os.path.join(downloadDir,dcmfolder + '.zip'))
  xnat.disconnect()
  return expdirlist


In [ ]:
xhost = 'https://pixnat.mskcc.org'
user = ''

#example of dictionary to describe the data you want to pull locally from XNAT,
# one entry per experiment, iterate over scans
scandict = [{'exam':
              {'proj':'ROBIN_MCT1',
                'subj':'25121-01',
                'exp':'',
                'scan_list':['501']
                }
            }]

dataPath = '/content/images'
os.makedirs(dataPath,exist_ok=True)

In [ ]:
folderList = getXNATData(xhost,user,scandict,dataPath)

##Import T2 DICOM to planC and save to NIfTI for segmentation

In [ ]:
from cerr import plan_container as pc

In [ ]:
# Import DICOM to planC

planC = pc.loadDcmDir(folderList[0])

In [ ]:
os.makedirs('/content/images/nifti', exist_ok=True)
planC.scan[0].saveNii('/content/images/nifti/scan6.nii')

## Install SMIT model via `model_installer`

In [ ]:
!cd /content && git clone https://github.com/cerr/model_installer.git

In [ ]:
os.chdir('/content/model_installer')
! source installer.sh -h

In [ ]:
from getpass import getpass
tok = getpass()

In [ ]:
! ./installer.sh -m 10 -p N -d /content -u {tok}

In [ ]:
uvpath = '/content/MR_Rectum_GTV_SMIT/.venv39'

In [ ]:
!uv venv --python-preference only-managed -p 3.9 {uvpath}

In [ ]:
%%bash
for rfile in `ls /content/MR_Rectum_GTV_SMIT/requirements/*`
do
  uv pip install -n -p /content/MR_Rectum_GTV_SMIT/.venv39 -r $rfile
done

In [ ]:
data_input = '/content/images/nifti'
seg_output = '/content/images/nifti'
load_weight = '/content/MR_Rectum_GTV_SMIT/trained_weights/model_final.pt'
!chmod -R 777 /content/MR_Rectum_GTV_SMIT

In [ ]:
os.chdir('/content/MR_Rectum_GTV_SMIT')
! uv run -p {uvpath} ./bash_run_SMIT_Segmentation.sh {data_input} {seg_output} {load_weight}

## Import NIfTI segmentation mask to planC

In [ ]:
from glob import glob

structNiiFileName = glob('/content/images/nifti/model*.nii')
niiFileName = structNiiFileName[0]
assocScanNum = 0
labelDict = {1: 'GTV'}
planC = pc.loadNiiStructure(niiFileName, assocScanNum, planC, labelDict)

## Visualize segmentation in Napari

In [ ]:
from cerr.viewer import showMplNb
scanNum = 0
struct_nums=[0]
showMplNb(planC=planC, scanNum=scanNum,
          structNums=struct_nums,
          windowCenter=-400, windowWidth=2000)